<img src="https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png" alt="Wherobots Logo" width="600">

## Electric Vehicle Charging Station Site Selection Analysis

This notebook demonstrates a workflow for identifying potential areas for new electric vehicle (EV) charging station development using WherobotsDB and WherobotsAI raster inference functionality. The workflow is based on:

* Identifying existing EV charging station infrastructure
* Proximity to retail stores as a proxy for demand, and
* Proximity to solar farms
    

Existing charging station infrastructure and retail store point of interest data is determined using public data sources, while existing solar farm infrastructure is identified using Wherobots AI raster inference. By using a machine learning model trained on satellite imagery we can identify solar farms as an input to the analysis.

**_NOTE: This notebook must be run in a GPU-enabled [Wherobots Cloud](https://cloud.wherobots.com/) Runtime for access to WherobotsAI Raster Inference functionality, which is a Professional Tier feature. Please [contact us](https://wherobots.com/contact/) for access._**

In [1]:
from sedona.spark import *
import os
import warnings
warnings.filterwarnings('ignore')

# specifies catalog called benchmark, the havasu catalog
# need to get from terminal
config = SedonaContext.builder() \
           .config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider", "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider") \
            .config("spark.driver.maxResultSize", "10g") \
           .config("spark.sql.catalog.benchmark.type", "hadoop") \
           .config("spark.sql.catalog.benchmark", "org.apache.iceberg.spark.SparkCatalog") \
           .config("spark.sql.catalog.benchmark.warehouse", "s3://wherobots-benchmark-prod/data/ml/") \
           .config("spark.sql.catalog.benchmark.io-impl", "org.apache.iceberg.aws.s3.S3FileIO").getOrCreate()
sedona = SedonaContext.create(config)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Identify Area Of Interest

We will use US Census Zip Code Tabulated Areas (ZCTA) to identify regions for potential EV charging station development. We will confine our analysis to the state of Arizona.

Note that we are using the `ST_Intersects` spatial predicate function to find ZCTAs that intersect with the border of Arizona rather than `ST_Contains`. This is because some ZCTAs extend beyond the border of Arizona and can lie within multiple states.

In [2]:
az_zips_df = sedona.sql("""
WITH arizona AS ( 
    SELECT localityArea.geometry AS geometry
    FROM wherobots_open_data.overture_2024_02_15.admins_locality locality 
    JOIN wherobots_open_data.overture_2024_02_15.admins_localityArea localityArea 
    ON locality.id = localityArea.localityId
    WHERE locality.names.primary = "Arizona" AND locality.localityType = "state" 
)

SELECT ST_Intersection(arizona.geometry, zta5.geometry) AS geometry, ZCTA5CE10 
FROM wherobots_pro_data.us_census.zipcode zta5, arizona
WHERE ST_Intersects(arizona.geometry, zta5.geometry)
""")

In [3]:
az_zips_df.createOrReplaceTempView("az_zta5")

In [4]:
az_zips_df.printSchema()

root
 |-- geometry: geometry (nullable = true)
 |-- ZCTA5CE10: string (nullable = true)



In [ ]:
SedonaKepler.create_map(az_zips_df, name="Arizona ZCTAs")

![Arizona ZCTAs](https://wherobots.com/wp-content/uploads/2024/06/AZ_ZCTA.png)

Next, we will identify existing EV charging infrastructure within each ZCTA as an input to our analysis.

## Existing EV Charging Infrastructure

Using data from [Open Charge Map](https://openchargemap.org/site) we calculate the number of EV charging stations in each ZCTA to give us a sense of existing EV charging infrastructure.


In [6]:
stations_df = sedona.read.format("geoparquet").load("s3://wherobots-examples/data/examples/openchargemap/world.parquet")

In [7]:
stations_df.createOrReplaceTempView("stations")

In [ ]:
SedonaKepler.create_map(stations_df.sample(0.01), name="EV Charging Stations")

![Existing EV chargers](https://wherobots.com/wp-content/uploads/2024/06/ev_chargers.png)

Count of existing EV charging stations per ZCTA.

In [9]:
az_stations_df = sedona.sql("""
SELECT COUNT(*) AS num, any_value(az_zta5.geometry) AS geometry, ZCTA5CE10
FROM stations JOIN az_zta5
WHERE ST_Intersects(az_zta5.geometry, stations.geometry)
GROUP BY ZCTA5CE10 
ORDER BY num DESC
""")

In [10]:
az_stations_df.createOrReplaceTempView("az_stations")

In [11]:
az_stations_df.count()

204

In [12]:
az_stations_df.cache().show()

+---+--------------------+---------+
|num|            geometry|ZCTA5CE10|
+---+--------------------+---------+
| 51|POLYGON ((-111.97...|    85281|
| 36|POLYGON ((-111.95...|    85251|
| 33|POLYGON ((-111.92...|    85260|
| 28|POLYGON ((-112.07...|    85004|
| 27|POLYGON ((-112.14...|    85226|
| 26|POLYGON ((-112.07...|    86336|
| 22|POLYGON ((-111.97...|    85054|
| 22|POLYGON ((-112.05...|    85016|
| 20|POLYGON ((-111.89...|    85286|
| 19|POLYGON ((-112.06...|    85034|
| 17|POLYGON ((-111.97...|    85254|
| 17|POLYGON ((-111.69...|    85212|
| 16|POLYGON ((-111.75...|    85206|
| 15|POLYGON ((-111.92...|    85250|
| 15|POLYGON ((-111.95...|    86001|
| 15|POLYGON ((-110.96...|    85719|
| 15|POLYGON ((-112.28...|    85305|
| 15|MULTIPOLYGON (((-...|    85282|
| 15|POLYGON ((-111.94...|    85248|
| 14|POLYGON ((-112.36...|    85323|
+---+--------------------+---------+
only showing top 20 rows



In [13]:
az_stations_df.printSchema()

root
 |-- num: long (nullable = false)
 |-- geometry: geometry (nullable = true)
 |-- ZCTA5CE10: string (nullable = true)



In [ ]:
SedonaKepler.create_map(az_stations_df, name="EV Chargers")

![](https://wherobots.com/wp-content/uploads/2024/06/ev_chargers_map.png)

## Arizona Retail Stores

Next, we'll use retail stores per ZCTA as a proxy for demand. ZCTAs with more retail stores indicate higher demand for EV chargers and EV charger users typically expect nearby amenities. Using the Overture Maps Foundation public point of interest data set we will find points of interest within each ZCTA using a spatial join operation and count the number of stores with a `GROUP BY`.

In [15]:
sedona.table("wherobots_open_data.overture_2024_02_15.places_place").count()

53622897

In [16]:
az_retail_df = sedona.sql("""
SELECT COUNT(*) AS num, any_value(az_zta5.geometry) AS geometry, ZCTA5CE10
FROM wherobots_open_data.overture_2024_02_15.places_place places 
JOIN az_zta5
WHERE ST_Intersects(az_zta5.geometry, places.geometry)
AND places.categories.main IN ("retail", "grocery_store", "restaurant") 
GROUP BY ZCTA5CE10 
ORDER BY num DESC
""")

In [17]:
az_retail_df.createOrReplaceTempView("az_retail")

In [18]:
az_retail_df.cache().show(5)

+---+--------------------+---------+
|num|            geometry|ZCTA5CE10|
+---+--------------------+---------+
| 50|POLYGON ((-111.92...|    85260|
| 49|POLYGON ((-111.97...|    85281|
| 39|POLYGON ((-112.07...|    85004|
| 38|POLYGON ((-111.04...|    85705|
| 37|POLYGON ((-112.14...|    85226|
+---+--------------------+---------+
only showing top 5 rows



In [ ]:
SedonaKepler.create_map(az_retail_df, name="Retail Stores")

![Retail stores](https://wherobots.com/wp-content/uploads/2024/06/retail_store_map.png)

## Combining Retail Stores & Existing EV Chargers

Before we apply WherobotsAI raster inference to identify solar farms in the area, we'll use existing EV chargers and retail stores to identify ZCTAs with high demand and low existing EV charging infrastructure by computing the ratio of retail stores to EV chargers in each ZCTA. We'll then normalize this value in the range 0.0 - 1.0 so we can treat this as part of our linear suitability model. A high opportunity score indicates an area where demand for EV chargers is unmet, relative to existing EV chargers in the area.


In [20]:
az_ratio = sedona.sql("""
WITH ratios AS (
    SELECT 
        coalesce(az_stations.num, 0) / coalesce(az_retail.num, 1) AS ratio, 
        coalesce(az_stations.geometry, az_retail.geometry) AS geometry, 
        coalesce(az_stations.ZCTA5CE10, az_retail.ZCTA5CE10) AS ZCTA5CE10
    FROM az_retail FULL OUTER JOIN az_stations
    ON az_retail.ZCTA5CE10 = az_stations.ZCTA5CE10
    WHERE az_retail.num > 1
    ORDER BY ratio DESC
),
min_max AS (
    SELECT MIN(ratio) AS min_val, MAX(ratio) AS max_val 
    FROM ratios
)

SELECT 
    (1 - ( (ratio - min_max.min_val ) / (min_max.max_val - min_max.min_val)   )) AS opportunity_score,
    geometry,
    ZCTA5CE10
FROM ratios, min_max
""")

In [21]:
az_ratio.createOrReplaceTempView("az_ratio")

In [22]:
az_ratio.cache().show()

+------------------+--------------------+---------+
| opportunity_score|            geometry|ZCTA5CE10|
+------------------+--------------------+---------+
|             0.824|POLYGON ((-111.92...|    85260|
|0.7224489795918367|POLYGON ((-111.97...|    85281|
|0.8085470085470086|POLYGON ((-112.07...|    85004|
|0.9228070175438596|POLYGON ((-111.04...|    85705|
|0.8054054054054054|POLYGON ((-112.14...|    85226|
|0.7405405405405405|POLYGON ((-111.95...|    85251|
|0.9185185185185185|POLYGON ((-112.23...|    85308|
| 0.853763440860215|POLYGON ((-111.97...|    85254|
|0.8709677419354839|POLYGON ((-110.96...|    85719|
| 0.956989247311828|MULTIPOLYGON (((-...|    85301|
|0.9733333333333334|MULTIPOLYGON (((-...|    85364|
|0.8755555555555555|POLYGON ((-112.03...|    85032|
|0.9356321839080459|POLYGON ((-111.86...|    85210|
|0.9632183908045977|POLYGON ((-112.16...|    85009|
|0.8620689655172413|MULTIPOLYGON (((-...|    85282|
|0.7904761904761904|POLYGON ((-112.05...|    85016|
|0.940740740

In [ ]:
SedonaKepler.create_map(az_ratio, name="Opportunity Score")

![](https://wherobots.com/wp-content/uploads/2024/06/opportunity_score.png)

ZCTAs with a high opportunity score are potential candidates for additional EV charging stations. The final input to our analysis is proximity to solar farms, which we will identify using WherobotsAI raster inference.

## WherobotsAI Raster Inference

[WherobotsAI](https://wherobots.com/wherobots-ai/) enables applying machine learning models to aerial imagery for segmentation, classification, and object detection. Our suitability analysis will leverage a segmentation model to identify solar farms from satellite imagery in our area of interest. We will use the opportunity score computed above to prioritize areas where raster inference should be applied, ensuring we are identifying solar farms only within ZCTAs with a high opportunity score.

The [outdb raster table](https://docs.wherobots.com/1.2.2/references/havasu/raster/out-db-rasters/) refers to Sentinel-2 images with low cloud cover during 2023 in Arizona. We've prepared this using WherbotsDB's raster processing capabilities.



In [24]:
columns_to_drop = ["x", "y", "product_type", "length"]
num_partitions = 32
solar_model_inputs_df = sedona.table("benchmark.db.solar_satlas_sentinel2_db").drop(*columns_to_drop).repartition(num_partitions)

In [25]:
solar_model_inputs_df.cache().show()

+--------------------+-------------+---------+-------------+--------------------+--------------+------------+--------------------+--------------------+
|            filename|surrogate_key|grid_code|constellation|            geometry|start_datetime|end_datetime|                path|        outdb_raster|
+--------------------+-------------+---------+-------------+--------------------+--------------+------------+--------------------+--------------------+
|sentinel-2-S2MSI2...|   1003289323|    12SUD|   sentinel-2|POLYGON ((-113.19...|      20230107|    20230206|s3a://wherobots-b...|OutDbGridCoverage...|
|sentinel-2-S2MSI2...|    -18933983|    11SPR|   sentinel-2|POLYGON ((-115.93...|      20231002|    20231027|s3a://wherobots-b...|OutDbGridCoverage...|
|sentinel-2-S2MSI2...|   -345599496|    12SUA|   sentinel-2|POLYGON ((-113.12...|      20230107|    20230122|s3a://wherobots-b...|OutDbGridCoverage...|
|sentinel-2-S2MSI2...|   -638119537|    12SXB|   sentinel-2|POLYGON ((-109.64...|      2

First, we search for scenes from our Sentinel-2 dataset that intersect with ZCTAs with a high opportunity score and filter for the latest images.

In [26]:
az_high_demand_with_scene_geom = sedona.sql(""" 
    WITH base as (
        SELECT s.filename, s.geometry as scene_geometry, s.outdb_raster as
        outdb_raster, z.ZCTA5CE10 as zip_code_name, z.geometry as zip_geometry, z.opportunity_score
        FROM benchmark.db.solar_satlas_sentinel2_db s, az_ratio z
        WHERE ST_Intersects(s.geometry, z.geometry)
        AND z.opportunity_score > 0.9 AND start_datetime > 20231001
    )
    SELECT DISTINCT filename, outdb_raster from base""").repartition(num_partitions)

In [27]:
%%time
print(az_high_demand_with_scene_geom.cache().count())

3000
CPU times: user 6.82 ms, sys: 3.89 ms, total: 10.7 ms
Wall time: 15 s


In [28]:
az_high_demand_with_scene_geom.createOrReplaceTempView("az_high_demand_with_scene")

Next, we apply our segmentation model using the `RS_SEGMENT` spatial SQL function.

In [29]:
model_id = 'solar-satlas-sentinel2'

sedona.sql(f"""
CREATE OR REPLACE TEMP VIEW segment_fields AS (
    SELECT
        outdb_raster, 
        RS_SEGMENT('{model_id}', outdb_raster) AS segment_result
    FROM
    az_high_demand_with_scene
)
""")

DataFrame[]

In [30]:
predictions_df = sedona.sql(f"""
SELECT
  outdb_raster, segment_result.*
FROM segment_fields
""")

In [31]:
%%time
print(predictions_df.cache().count())

3000
CPU times: user 63.4 ms, sys: 8.26 ms, total: 71.7 ms
Wall time: 5min 12s


In [32]:
predictions_df.show()
predictions_df.createOrReplaceTempView("predictions_df")

+--------------------+--------------------+-----------------+
|        outdb_raster|    confidence_array|        class_map|
+--------------------+--------------------+-----------------+
|OutDbGridCoverage...|[0.5000004, 0.5, ...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50000495, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50000507, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50000054, 0.5,...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000337, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000249, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50000465, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5002269, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5192978, 0.508...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000099, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50000167, 0.50...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.5000375, 0.500...|{Solar Farm -> 1}|
|OutDbGridCoverage...|[0.50001484, 0.50...|{Solar Farm -> 1}|
|OutDbGr

After generating predictions where each pixel has an associated predicted class and confidence score in the raster data, we georeference the predictions and convert to vector geometries using `RS_SEGMENT_TO_GEOMS`.

In [33]:
predictions_polys_df = sedona.sql("""
    WITH t AS (
        SELECT RS_SEGMENT_TO_GEOMS(outdb_raster, confidence_array, array(1), class_map, 0.65) result
        FROM predictions_df
    )
    SELECT result.* FROM t
""")

In [34]:
predictions_polys_df.createOrReplaceTempView("predictions_polys")

predictions_polys_df = sedona.sql("""
    SELECT
        class_name[0] AS class, average_pixel_confidence_score[0] AS avg_confidence_score, ST_SetSRID(ST_Collect(geometry), 4326) AS geometry
    FROM
        predictions_polys
""").filter("ST_IsEmpty(geometry) = False")

In [35]:
predictions_polys_df.cache().count()

132

In [36]:
predictions_polys_df.createOrReplaceTempView("predictions_polys")

In [37]:
predictions_polys_df.show()

+----------+--------------------+--------------------+
|     class|avg_confidence_score|            geometry|
+----------+--------------------+--------------------+
|Solar Farm|    0.68480470445421|MULTIPOLYGON (((-...|
|Solar Farm|  0.7128666241963705|MULTIPOLYGON (((-...|
|Solar Farm|  0.7266481673832252|MULTIPOLYGON (((-...|
|Solar Farm|  0.7276440286964075|MULTIPOLYGON (((-...|
|Solar Farm|  0.6999729461967945|MULTIPOLYGON (((-...|
|Solar Farm|  0.7246828334672111|MULTIPOLYGON (((-...|
|Solar Farm|  0.7185474231543131|MULTIPOLYGON (((-...|
|Solar Farm|  0.7282067544000932|MULTIPOLYGON (((-...|
|Solar Farm|  0.7255745135752941|MULTIPOLYGON (((-...|
|Solar Farm|  0.6694437503814697|MULTIPOLYGON (((-...|
|Solar Farm|   0.729298093351492|MULTIPOLYGON (((-...|
|Solar Farm|  0.7281817384064198|MULTIPOLYGON (((-...|
|Solar Farm|  0.7299639159883475|MULTIPOLYGON (((-...|
|Solar Farm|  0.7264729229859261|MULTIPOLYGON (((-...|
|Solar Farm|  0.7253007416098843|MULTIPOLYGON (((-...|
|Solar Far

In [ ]:
SedonaKepler.create_map(predictions_polys_df, name="Detected Solar Farms")

![](https://wherobots.com/wp-content/uploads/2024/06/solar_farm_detection.png)

## Compute Final Suitability Score

Our final suitability score for each ZCTA will be based on the opportunity score computed above and the total area of all solar farms detected in each ZCTA using WherobotsAI Raster Inference.

First, we compute the area of solar farm predictions in each ZCTA and then normalize in a range of 0.0 - 1.0, where 1.0 represents the score for the ZCTA with the most area of solar farms detected.

In [39]:
az_solar_zip_codes = sedona.sql("""
WITH solar AS (
    SELECT 
        ST_AreaSpheroid(ST_Union_Aggr(ST_SetSRID(predictions_polys.geometry, 4326))) / 1000000 AS solar_area, 
        any_value(az_zta5.geometry) AS geometry, 
        ZCTA5CE10
    FROM predictions_polys JOIN az_zta5
    WHERE ST_Intersects(az_zta5.geometry, predictions_polys.geometry)
    GROUP BY ZCTA5CE10 
    ORDER BY solar_area DESC
),

min_max AS (
    SELECT MIN(solar_area) AS min_val, MAX(solar_area) AS max_val
    FROM solar
)

SELECT (solar_area - min_max.min_val) / (min_max.max_val - min_max.min_val) AS solar_score,
    geometry,
    ZCTA5CE10
FROM solar, min_max
""")


In [40]:
az_solar_zip_codes.show()

+--------------------+--------------------+---------+
|         solar_score|            geometry|ZCTA5CE10|
+--------------------+--------------------+---------+
|  0.7382013549912736|POLYGON ((-113.61...|    85333|
|9.850013319116988E-4|MULTIPOLYGON (((-...|    85364|
|                 0.0|POLYGON ((-109.96...|    85625|
| 0.03738713953780777|MULTIPOLYGON (((-...|    85365|
|0.002464618157075...|POLYGON ((-110.03...|    85610|
|0.015369601008682895|POLYGON ((-114.34...|    85356|
|1.848446564826133...|POLYGON ((-112.36...|    85323|
|0.001786720384544...|POLYGON ((-112.48...|    86301|
|0.001047683730417...|POLYGON ((-112.07...|    86336|
|0.002295361518856...|POLYGON ((-112.20...|    85304|
|0.061654116143163185|POLYGON ((-110.00...|    86033|
|4.107613473448007...|POLYGON ((-112.49...|    86046|
|0.016804868783420225|POLYGON ((-112.40...|    85309|
|0.021582954348305126|POLYGON ((-113.33...|    86321|
| 0.03966279616289762|POLYGON ((-112.54...|    86323|
|0.001786720384544...|POLYGO

In [41]:
az_solar_zip_codes.count()

54

In [42]:
az_solar_zip_codes.createOrReplaceTempView("az_solar_zip_codes")

In [ ]:
SedonaKepler.create_map(az_solar_zip_codes, name="Solar Scores")

Then, to compute our final suitability score we combine the opportunity score and our solar area score together.

In [44]:
# join az_ratio with az_solar_zip codes

final_az_scores = sedona.sql("""
SELECT 
   opportunity_score + solar_score AS score, 
    az_solar_zip_codes.ZCTA5CE10 AS ZCTA5CE10,
    az_solar_zip_codes.geometry AS geometry
FROM az_ratio
JOIN az_solar_zip_codes
WHERE az_ratio.ZCTA5CE10 = az_solar_zip_codes.ZCTA5CE10
ORDER BY score DESC
""")

In [45]:
final_az_scores.cache().show()

+------------------+---------+--------------------+
|             score|ZCTA5CE10|            geometry|
+------------------+---------+--------------------+
|               2.0|    85354|POLYGON ((-113.33...|
|1.3007179799217377|    85326|POLYGON ((-112.77...|
|1.0771901371538422|    85325|POLYGON ((-114.21...|
|1.0716857806433455|    85344|POLYGON ((-114.13...|
|1.0616541161431632|    86033|POLYGON ((-110.00...|
|  1.04670244270891|    85367|POLYGON ((-114.49...|
|1.0396627961628977|    86323|POLYGON ((-112.54...|
| 1.021582954348305|    86321|POLYGON ((-113.33...|
|1.0183157905636342|    86401|POLYGON ((-114.14...|
|1.0161242130880033|    86426|POLYGON ((-114.61...|
|1.0131786984855902|    85348|POLYGON ((-114.13...|
|1.0104760374649069|    86413|POLYGON ((-114.67...|
|1.0029112572626941|    85392|POLYGON ((-112.35...|
|1.0012934689228656|    85632|POLYGON ((-109.04...|
|1.0010569887056573|    85349|POLYGON ((-114.81...|
|1.0003440863199857|    86052|POLYGON ((-112.64...|
|           

In [46]:
final_az_scores.createOrReplaceTempView("final_scores")

## Generate Final Inputs

In [47]:
# Find ev chargers in priority areas

final_az_chargers = sedona.sql("""
SELECT stations.geometry, stations.id AS station_id
FROM stations, final_scores
WHERE ST_Contains(final_scores.geometry, stations.geometry)
""")

final_az_chargers.cache().count()

141

In [48]:
# Find retail stores in priority areas

final_retail = sedona.sql("""
SELECT places.geometry 
FROM wherobots_open_data.overture_2024_02_15.places_place places
JOIN final_scores
WHERE ST_Contains(final_scores.geometry, places.geometry) AND places.categories.main = "retail"
""")

final_retail.cache().count()


111

In [49]:
# Find solar farms in priority areas

final_solar = sedona.sql("""
SELECT predictions_polys.class, predictions_polys.avg_confidence_score, predictions_polys.geometry
FROM predictions_polys
JOIN final_scores
WHERE ST_Intersects(final_scores.geometry, predictions_polys.geometry)
""")

final_solar.cache().count()


73

## Final Suitability Analysis

We can visualize the final results of this suitability analysis as a choropleth map, including each individual component of the analysis: detected solar farms, retail stores, and existing EV charging stations.

In [ ]:
final_map = SedonaKepler.create_map(final_az_scores, name="Suitability Results")
SedonaKepler.add_df(final_map, final_solar, name="Solar Farms")
SedonaKepler.add_df(final_map, final_retail, name="Retail Stores")
SedonaKepler.add_df(final_map, final_az_chargers, name="Existing EV Chargers")
final_map

![](https://wherobots.com/wp-content/uploads/2024/06/final_analysis.png)

## Write Analysis Results As PMTiles

Vector tiles are frequently used for rendering map data in web maps, mobile apps, and desktop GIS software. Wherobots can generate vector tiles in the PMTiles format. [See the documentation](https://docs.wherobots.com/latest/tutorials/wherobotsdb/tile-generation/tile-generation/) for more information on generating vector tiles at scale. 


In [51]:
from wherobots import vtiles
from pyspark.sql.functions import lit


# Define paths to save PMTiles files in S3
zip_tiles_path = os.getenv("USER_S3_PATH") + "final_az_suitability.pmtiles"
ev_chargers_path = os.getenv("USER_S3_PATH") + "final_az_chargers.pmtiles"
retail_path = os.getenv("USER_S3_PATH") + "final_retail.pmtiles"
solar_path = os.getenv("USER_S3_PATH") + "final_solar.pmtiles"

# Add "layer" column
final_az_scores_layers = final_az_scores.withColumn("layer", lit('Suitability Results'))
final_az_chargers_layers = final_az_chargers.withColumn("layer", lit('Existing Chargers'))
final_az_retail_layers = final_retail.withColumn("layer", lit("Retail stores"))
final_az_solar_layers = final_solar.withColumn("layer", lit("Solar Farms"))

# Generate and write PMTiles
tiles_df = vtiles.generate(final_az_scores_layers)
vtiles.write_pmtiles(tiles_df, zip_tiles_path, features_df=final_az_scores_layers)

charger_tiles_df = vtiles.generate(final_az_chargers_layers)
vtiles.write_pmtiles(charger_tiles_df, ev_chargers_path, features_df=final_az_chargers_layers)

retail_tiles_df = vtiles.generate(final_az_retail_layers)
vtiles.write_pmtiles(retail_tiles_df, retail_path, features_df=final_az_retail_layers)

solar_tiles_df = vtiles.generate(final_az_solar_layers)
vtiles.write_pmtiles(solar_tiles_df, solar_path, features_df=final_az_solar_layers)


24/06/12 16:01:26 WARN DAGScheduler: Broadcasting large task binary with size 1662.6 KiB
24/06/12 16:03:51 WARN DAGScheduler: Broadcasting large task binary with size 1672.2 KiB


## Visualize PMTiles

Now that we've generated and saved vector tiles for each layer we can quickly visualize them using `vtiles.show_pmtiles` which uses Leafmap to render our tiles. Optionally, we can also define styles to be applied to each layer.

In [ ]:
from wherobots.tools.utility.s3_utils import get_signed_url

tiles_config = [
    {
        "s3_uri": zip_tiles_path,
        "name": "Suitability",
        "style": {
            'version': 8,
            'sources': {
                'source': {
                    'type': 'vector',
                    'url': 'pmtiles://' + get_signed_url(zip_tiles_path),
                    'attribution': 'PMTiles'
                }
            },
         'layers': [
              {
                  'id': 'Suitability Results_fill',
                  'source': 'source',
                  'source-layer': 'Suitability Results',
                  'type': 'fill',
                  'paint': {'fill-color': 'lightblue', 'fill-opacity': 0.5},
                  'filter': ['==', ['geometry-type'], 'Polygon']
              }
         ]
        }
    },
    {
        "s3_uri": ev_chargers_path,
        "style": {
            'version': 8,
            'sources': {
                'source': {
                    'type': 'vector',
                    'url': 'pmtiles://' + get_signed_url(ev_chargers_path),
                    'attribution': 'PMTiles'
                }
            },
            'layers': [
                {
                    'id': 'Existing Chargers_point',
                    'source': 'source',
                    'source-layer': 'Existing Chargers',
                    'type': 'circle',
                    'paint': {'circle-color': 'purple', 'circle-radius': 5},
                    'filter': ['==', ['geometry-type'], 'Point']}]}
                },
    {
        "s3_uri": retail_path
    },
    {
        "s3_uri": solar_path
    }
]

vtiles.show_pmtiles(tiles_config)

![Visualizing PMTiles with Leafmap](https://wherobots.com/wp-content/uploads/2024/06/leafmap_pmtiles_suitability.png)

## Writing to Iceberg Tables And The Wherobots Catalog

We can write the results of our analysis to Iceberg tables using SQL, which can then be accessed by other users or analytics applications, including via the [Spatial SQL API using one of the WherobotsDB language drivers (Python or JDBC)](https://docs.wherobots.com/latest/develop/spatial-sql-api/). Wherobots supports an extension of Apache Iceberg called [Havasu](https://docs.wherobots.com/latest/develop/storage-management/dataset-catalog/) which adds functionality and optimizations for vector and raster geospatial data. By default the `wherobots` catalog is configured to be used with your Wherobots Cloud account.

We'll create a new table `wherobots.suitability.ev_chargers` which will contain the results of our analysis, including the final suitability score and associated polygon geometry.

In [53]:
sedona.sql("CREATE NAMESPACE IF NOT EXISTS wherobots.suitability")
sedona.sql("DROP TABLE IF EXISTS wherobots.suitability.ev_chargers")
final_az_scores.writeTo("wherobots.suitability.ev_chargers").create()

In [54]:
sedona.table("wherobots.suitability.ev_chargers").show()

+------------------+---------+--------------------+
|             score|ZCTA5CE10|            geometry|
+------------------+---------+--------------------+
|               2.0|    85354|POLYGON ((-113.33...|
|1.3007179799217377|    85326|POLYGON ((-112.77...|
|1.0771901371538422|    85325|POLYGON ((-114.21...|
|1.0716857806433455|    85344|POLYGON ((-114.13...|
|1.0616541161431632|    86033|POLYGON ((-110.00...|
|  1.04670244270891|    85367|POLYGON ((-114.49...|
|1.0396627961628977|    86323|POLYGON ((-112.54...|
| 1.021582954348305|    86321|POLYGON ((-113.33...|
|1.0183157905636342|    86401|POLYGON ((-114.14...|
|1.0161242130880033|    86426|POLYGON ((-114.61...|
|1.0131786984855902|    85348|POLYGON ((-114.13...|
|1.0104760374649069|    86413|POLYGON ((-114.67...|
|1.0029112572626941|    85392|POLYGON ((-112.35...|
|1.0012934689228656|    85632|POLYGON ((-109.04...|
|1.0010569887056573|    85349|POLYGON ((-114.81...|
|1.0003440863199857|    86052|POLYGON ((-112.64...|
|           

## Querying Data Via QGIS

Now that we've saved the results of our suitability analysis in a table we can query this data in external applications via the Wherobots Spatial SQL API. For example, we may want to pull this data into an application like QGIS to integrate with other data workflows. We can do this using a Python script in QGIS and making use of the [Wherobots Python driver.](https://docs.wherobots.com/latest/develop/spatial-sql-api/#python-db-api-driver)


```
pip install wherobots-python-dpapi
```

In Wherobots Cloud, [create a new API key.](https://docs.wherobots.com/latest/get-started/api-keys/)

Then in the QGIS Python console we can use the Wherobots Python driver to run Spatial SQL queries, including accessing Havasu tables. In this example we fetch our suitability analysis results, convert to a GeoDataFrame, then use the PyQGIS API to add the data to a new vector layer in QGIS.


```python
from wherobots.db import connect
from wherobots.db.region import Region
from wherobots.db.runtime import Runtime
import geopandas 
from shapely import wkt


with connect(
        api_key=WBC_KEY,
        runtime=Runtime.SEDONA,
        region=Region.AWS_US_WEST_2,
        host="api.cloud.wherobots.com"
) as conn:
    curr = conn.cursor()
    curr.execute("""
    SELECT *
    FROM wherobots.suitability.ev_chargers
    """)
    results = curr.fetchall()
    print(results)
    
results['geometry'] = results.geometry.apply(wkt.loads)
gdf = geopandas.GeoDataFrame(results, crs='EPSG:4326',geometry='geometry')

def add_geodataframe_to_layer(geodataframe, layer_name):
    # Create a new memory layer
    layer = QgsVectorLayer(geodataframe.to_json(), layer_name, "ogr")
    
   
    
    # Add the layer to the QGIS project
    QgsProject.instance().addMapLayer(layer)
    
add_geodataframe_to_layer(gdf, "Suitability Analysis")

```

![](https://wherobots.com/wp-content/uploads/2024/06/qgis_suitability_analysis.png)


You can read more about the Wherobots Spatial SQL API, including examples in Python, Java, scheduling jobs via Airflow, and using SQL IDEs like Harlequin in [the Wherobots documentation.](https://docs.wherobots.com/latest/develop/spatial-sql-api/)
